In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [5]:
def my_neo4j_wipe_out_mst_relationships():
    "wipe out mst relationships"
    
    query = "match (node)-[relationship:MST]->() delete relationship"
    session.run(query)

### Louvain Modularity

In [6]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [7]:
query = """

CALL gds.louvain.stream('ds_graph', { relationshipWeightProperty: 'weight' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community
ORDER BY community, name ASC

"""

my_neo4j_run_query_pandas(query)


,name,community
0,blue Montgomery Street,0
1,blue Powell Street,0
2,green Montgomery Street,0
3,green Powell Street,0
4,red Montgomery Street,0
...,...,...
209,orange Hayward,211
210,orange Milpitas,211
211,orange South Hayward,211
212,orange Union City,211


In [8]:
#Define dataframe

df = my_neo4j_run_query_pandas(query)

In [9]:
#Remove arrive and depart nodes

df = df[df["name"].str.contains("arrive") == False] 
df = df[df["name"].str.contains("depart") == False] 

In [10]:
#Confirm quantity of communities and quantity of nodes within each community

df.community.value_counts()

211    16
43     14
204    14
188    11
173    10
0       8
2       8
165     8
196     8
17      7
8       6
33      4
Name: community, dtype: int64

In [12]:
df[df['community'] == 211]

,name,community
198,green Bay Fair,211
199,green Berryessa,211
200,green Fremont,211
201,green Hayward,211
202,green Milpitas,211
203,green South Hayward,211
204,green Union City,211
205,green Warm Springs,211
206,orange Bay Fair,211
207,orange Berryessa,211


In [13]:
df[df['community'] == 43]

,name,community
39,orange 19th Street,43
40,orange Ashby,43
41,orange Downtown Berkeley,43
42,orange MacArthur,43
43,red 19th Street,43
44,red Ashby,43
45,red Downtown Berkeley,43
46,red MacArthur,43
47,yellow 19th Street,43
48,yellow Lafayette,43


In [14]:
df[df['community'] == 204]

,name,community
184,blue Coliseum,204
185,blue Fruitvale,204
186,blue Lake Merritt,204
187,blue San Leandro,204
188,gray Coliseum,204
189,gray OAK,204
190,green Coliseum,204
191,green Fruitvale,204
192,green Lake Merritt,204
193,green San Leandro,204


In [15]:
df[df['community'] == 188]

,name,community
165,blue Embarcadero,188
166,blue West Oakland,188
167,green Embarcadero,188
168,green West Oakland,188
169,orange 12th Street,188
170,red 12th Street,188
171,red Embarcadero,188
172,red West Oakland,188
173,yellow 12th Street,188
174,yellow Embarcadero,188


In [16]:
df[df['community'] == 173]

,name,community
155,blue Balboa Park,173
156,blue Daly City,173
157,green Balboa Park,173
158,green Daly City,173
159,red Balboa Park,173
160,red Colma,173
161,red Daly City,173
162,yellow Balboa Park,173
163,yellow Colma,173
164,yellow Daly City,173


In [17]:
df[df['community'] == 0]

,name,community
0,blue Montgomery Street,0
1,blue Powell Street,0
2,green Montgomery Street,0
3,green Powell Street,0
4,red Montgomery Street,0
5,red Powell Street,0
6,yellow Montgomery Street,0
7,yellow Powell Street,0


In [18]:
df[df['community'] == 2]

,name,community
8,orange El Cerrito Plaza,2
9,orange El Cerrito del Norte,2
10,orange North Berkeley,2
11,orange Richmond,2
12,red El Cerrito Plaza,2
13,red El Cerrito del Norte,2
14,red North Berkeley,2
15,red Richmond,2


In [19]:
df[df['community'] == 165]

,name,community
147,blue 16th Street Mission,165
148,blue Civic Center,165
149,green 16th Street Mission,165
150,green Civic Center,165
151,red 16th Street Mission,165
152,red Civic Center,165
153,yellow 16th Street Mission,165
154,yellow Civic Center,165


In [20]:
df[df['community'] == 196]

,name,community
176,blue 24th Street Mission,196
177,blue Glen Park,196
178,green 24th Street Mission,196
179,green Glen Park,196
180,red 24th Street Mission,196
181,red Glen Park,196
182,yellow 24th Street Mission,196
183,yellow Glen Park,196


In [21]:
df[df['community'] == 17]

,name,community
22,red Millbrae,17
23,red SFO,17
24,red San Bruno,17
25,red South San Francisco,17
26,yellow SFO,17
27,yellow San Bruno,17
28,yellow South San Francisco,17


In [22]:
df[df['community'] == 8]

,name,community
16,yellow Antioch,8
17,yellow Concord,8
18,yellow North Concord,8
19,yellow Pittsburg,8
20,yellow Pittsburg Center,8
21,yellow Pleasant Hill,8


In [23]:
df[df['community'] == 33]

,name,community
29,blue Bay Fair,33
30,blue Castro Valley,33
31,blue Dublin,33
32,blue West Dublin,33


In [24]:
stations = pd.read_csv('stations-Copy1.csv')

In [25]:
# new data frame with split value columns
new = df["name"].str.split(" ", n=1, expand=True)
 
# making separate first name column from new data frame
df["line color"] = new[0]
 
# making separate last name column from new data frame
df["station"] = new[1]
 
# Dropping old Name columns
df.drop(columns=["name"], inplace=True)
 
# df display
df

,community,line color,station
0,0,blue,Montgomery Street
1,0,blue,Powell Street
2,0,green,Montgomery Street
3,0,green,Powell Street
4,0,red,Montgomery Street
...,...,...,...
209,211,orange,Hayward
210,211,orange,Milpitas
211,211,orange,South Hayward
212,211,orange,Union City


In [30]:
df = df.merge(stations, how='left', left_on='station', right_on='station')

In [33]:
df = df.drop(['transfer_time'],axis=1)

In [35]:
df = df.drop(['line color'],axis=1)

In [39]:
df = df.drop_duplicates()

In [40]:
df

,community,station,latitude,longitude
0,0,Montgomery Street,37.789355,-122.401942
1,0,Powell Street,37.784000,-122.408000
8,2,El Cerrito Plaza,37.902694,-122.298968
9,2,El Cerrito del Norte,37.925183,-122.316939
10,2,North Berkeley,37.873915,-122.282552
11,2,Richmond,37.936811,-122.353095
16,8,Antioch,37.996281,-121.783404
17,8,Concord,37.973745,-122.029127
18,8,North Concord,38.003273,-122.024599
19,8,Pittsburg,38.018869,-121.944208


In [41]:
df.to_csv('community_list.csv')